In [1]:
import os
import numpy as np
import shutil
import tensorflow as tf
from typing import Tuple

from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

from tf_utils.Cifar10DataAdvanced import CIFAR10

In [2]:
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
LOGS_DIR = os.path.abspath('C:/Selbststudium/Udemy/Udemy_Tensorflow/logs/Cifar/')
if not os.path.exists(LOGS_DIR):
    os.mkdir(LOGS_DIR)

In [4]:
def build_model(
    img_shape: Tuple[int, int, int],
    num_classes: int,
    optimizer: tf.keras.optimizers.Optimizer,
    learning_rate: float,
    filter_block_1: int,
    kernel_size_block_1: int,
    filter_block_2: int,
    kernel_size_block_2: int,
    filter_block_3: int,
    kernel_size_block_3: int,
    filter_block_4: int,
    kernel_size_block_4: int,
    dense_layer_size: int,
) -> Model:
    input_img = Input(shape=img_shape)

    x = Conv2D(filters=filter_block_1, kernel_size=kernel_size_block_1, padding='same')(input_img)
    x = Activation('relu')(x)
    x = Conv2D(filters=filter_block_1, kernel_size=kernel_size_block_1, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(filters=filter_block_2, kernel_size=kernel_size_block_2, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=filter_block_2, kernel_size=kernel_size_block_2, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(filters=filter_block_3, kernel_size=kernel_size_block_3, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=filter_block_3, kernel_size=kernel_size_block_3, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(filters=filter_block_4, kernel_size=kernel_size_block_4, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=filter_block_4, kernel_size=kernel_size_block_4, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Flatten()(x)
    x = Dense(units=dense_layer_size)(x)
    x = Activation('relu')(x)
    x = Dense(units=num_classes)(x)
    y_pred = Activation('softmax')(x)

    # Jetzt muss noch ein Modell Objekt mit eben obiger Struktur erstellt werden
    model = Model(
        inputs = [input_img],
        outputs = [y_pred]
    )
    
    opt = optimizer(learning_rate=learning_rate)
    
    model.compile(
        loss='categorical_crossentropy', # wird bei Kategorie-Problemen mit mehr als 2 Klassen genommen
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

In [5]:
data = CIFAR10()

train_dataset = data.get_train_set()
val_dataset = data.get_val_set()
test_dataset = data.get_test_set()

img_shape = data.img_shape
num_classes = data.num_classes

epochs = 100
batch_size = 256

# Hier nun die Parameter des Models der GridSearch eintragen
# param_grid={...} wird hier nicht mehr benötigt
# Best model params
optimizer = RMSprop
learning_rate = 0.001
filter_block1 = 32
kernel_size_block1 = 3
filter_block2 = 64
kernel_size_block2 = 3
filter_block3 = 64
kernel_size_block3 = 7
filter_block4 = 256
kernel_size_block4 = 5
dense_layer_size = 512



In [6]:
model = build_model(
    img_shape,
    num_classes,
    optimizer,
    learning_rate,
    filter_block1,
    kernel_size_block1,
    filter_block2,
    kernel_size_block2,
    filter_block3,
    kernel_size_block3,
    filter_block4,
    kernel_size_block4,
    dense_layer_size,
)

model_log_dir = os.path.join(LOGS_DIR, "modelBest_imp1")
# Best --- Loss: 0.0891 - accuracy: 0.9751 - val_loss: 2.3750 - val_accuracy: 0.7415

tb_callback = TensorBoard(
    log_dir=model_log_dir,
    histogram_freq=0,
    profile_batch=0
)

es_callback = EarlyStopping(
    monitor="val_accuracy",
    patience=30,
    verbose=1,
    restore_best_weights=True,
    min_delta=0.0005
)

model.fit(
    train_dataset,
    verbose=1,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[tb_callback],
    validation_data=val_dataset,
)

val_accuracy = model.evaluate(
    test_dataset, 
    batch_size=batch_size,
    verbose=0
)

Epoch 1/100
262/262 [==============================] - 22s 65ms/step - loss: 2.3420 - accuracy: 0.1338 - val_loss: 2.3710 - val_accuracy: 0.1288
Epoch 2/100
262/262 [==============================] - 15s 59ms/step - loss: 1.8938 - accuracy: 0.2878 - val_loss: 1.8960 - val_accuracy: 0.3193
Epoch 3/100
262/262 [==============================] - 15s 59ms/step - loss: 1.5633 - accuracy: 0.4257 - val_loss: 1.3592 - val_accuracy: 0.5094
Epoch 4/100
262/262 [==============================] - 15s 57ms/step - loss: 1.3162 - accuracy: 0.5267 - val_loss: 1.2208 - val_accuracy: 0.5673
Epoch 5/100
262/262 [==============================] - 15s 58ms/step - loss: 1.1274 - accuracy: 0.5999 - val_loss: 1.0482 - val_accuracy: 0.6322
Epoch 6/100
262/262 [==============================] - 15s 58ms/step - loss: 0.9677 - accuracy: 0.6624 - val_loss: 1.0107 - val_accuracy: 0.6455
Epoch 7/100
262/262 [==============================] - 15s 58ms/step - loss: 0.8211 - accuracy: 0.7162 - val_loss: 0.9219 - val_ac